# Using Structured Response
Integrating `TypedDict` or `Pydantic` schema into LLM will allow us to get final response in structured format, making it easier to work with the data.

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os

if "GOOGLE_API_KEY" not in os.environ:
    raise ValueError("Missing GOOGLE_API_KEY environment variable")
# Create the LLM instance
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0.1,
    max_output_tokens=1024,
)

In [3]:
# Without Structured Response
from langchain_core.messages import HumanMessage, SystemMessage
llm.invoke([
    SystemMessage(content="You are a helpful assistant that provides information about users and their likes. Format it like <UserName>: <Like 1>, <Like 2>"),
    HumanMessage(content="I am Mohit and I like Nolan's movies. Also love to play volleyball.")
])

AIMessage(content="Mohit: Nolan's movies, Volleyball", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id='run--db795092-6674-4e7d-99eb-14bba4c413a6-0', usage_metadata={'input_tokens': 49, 'output_tokens': 9, 'total_tokens': 58, 'input_token_details': {'cache_read': 0}})

## TypedDict
Convert each class objects into a Python Dictionary

In [ ]:
from typing_extensions import TypedDict, Annotated

class UserLikes(TypedDict):
    user: Annotated[str, "Name of the user"]
    likes: Annotated[list[str], "List of things the user likes"]

{'likes': ["Nolan's movies", 'volleyball'], 'user': 'Mohit'}

In [ ]:
llm_with_structured_response = llm.with_structured_output(UserLikes)

llm_with_structured_response.invoke([
    SystemMessage(content="You are a helpful assistant that provides information about users and their likes. Format it like <UserName>: <Like 1>, <Like 2>"),
    HumanMessage(content="I am Mohit and I like Nolan's movies. Also love to play volleyball.")
])

# Pydantic
More sophisticated data typing class.

In [5]:
from pydantic import BaseModel, Field

class UserLikesModel(BaseModel):
    user: str = Field(..., description="Name of the user")
    likes: list[str] = Field(..., description="List of things the user likes")

In [6]:
llm_with_structured_response = llm.with_structured_output(UserLikesModel)

llm_with_structured_response.invoke([
    SystemMessage(content="You are a helpful assistant that provides information about users and their likes. Format it like <UserName>: <Like 1>, <Like 2>"),
    HumanMessage(content="I am Mohit and I like Nolan's movies. Also love to play volleyball.")
])

UserLikesModel(user='Mohit', likes=["Nolan's movies", 'volleyball'])

# Trustcalls
![trustcalls](https://raw.githubusercontent.com/hinthornw/trustcall/8c7312b591542cf9349ce00d2811e890ea6eef91/_static/cover.png)

In [27]:
from trustcall import create_extractor
from pydantic import BaseModel, Field

class UserLikesModel(BaseModel):
    user: str = Field(..., description="Name of the user")
    likes: list[str] = Field(..., description="List of things the user likes")
    duckducksearches: list[str] = Field([], description="List of latest news for each likes by DuckDuckGo Search")

extractor = create_extractor(llm=llm, tools=[UserLikesModel], tool_choice="any")
resp = extractor.invoke([
    SystemMessage(content="You are a helpful assistant that provides information about users and their likes. Format it like <UserName>: <Like 1>, <Like 2>"),
    HumanMessage(content="I am Mohit and I like Nolan's movies. Also love to play volleyball. Get me latest news on my interests")
])
resp


{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'UserLikesModel', 'arguments': '{"duckducksearches": ["latest news on Nolan\'s movies", "latest news on volleyball"], "user": "Mohit", "likes": ["Nolan\'s movies", "volleyball"]}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id='run--466a2f85-c37a-44ea-8544-e4ece2a6c8c8-0', tool_calls=[{'name': 'UserLikesModel', 'args': {'duckducksearches': ["latest news on Nolan's movies", 'latest news on volleyball'], 'user': 'Mohit', 'likes': ["Nolan's movies", 'volleyball']}, 'id': '92f3cddd-c7cd-45f3-a063-863c8f5c545d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 144, 'output_tokens': 49, 'total_tokens': 193, 'input_token_details': {'cache_read': 0}})],
 'responses': [UserLikesModel(user='Mohit', likes=["Nolan's movies", 'volleyball'], duckducksearches=["latest news on Nolan's movies

In [28]:
resp["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  UserLikesModel (92f3cddd-c7cd-45f3-a063-863c8f5c545d)
 Call ID: 92f3cddd-c7cd-45f3-a063-863c8f5c545d
  Args:
    duckducksearches: ["latest news on Nolan's movies", 'latest news on volleyball']
    user: Mohit
    likes: ["Nolan's movies", 'volleyball']


In [29]:
resp["responses"][-1]

UserLikesModel(user='Mohit', likes=["Nolan's movies", 'volleyball'], duckducksearches=["latest news on Nolan's movies", 'latest news on volleyball'])